In [1]:
import os 
import json
import numpy as np
import descarteslabs as dl 
from pprint import pprint

## A few functions we want to run over a large area.

In [2]:
def mask_water(raster):
    shape = raster.shape
    length = raster.size
    
    # Reshape the matrix to a vector
    x = raster.reshape(length)
    
    # Slice every 4th element
    # This selects all the nir values from the vector 
    y = x[0::4]
       
    # Mask the output if less than 60 for NIR 
    # Create a vector of the same size as the NIR vector with all vals 60 for comparison
    sixty = np.ones(len(y))*60
    
    # Create a vector of boolean values, true when NIR is less than 60     
    z = y < sixty
       
    # Multiply by 4  - this will extend the bool vector by copying the value 4 times in place
    a = np.repeat(z, 4)
    
    # Compare the mask to the original raster vector: Returns the value where where false, and '--' where mask is true
    b = np.ma.masked_array(x,a)

    # Give masked nums the value of 0 
    b = np.ma.filled(b,0)
    
    # Push the output vector into its original 3D shape
    c = b.reshape(shape)
    return c

In [3]:
def get_land_area(image, area_per_pixel):
    # Flatten the rows, columns,and bands into a single array.      
    length =image.size
    flattened_image = image.reshape(length)
    
    # Select every fourth element, the fourth being the alpha band 
    # with a value of 0 for no valid data and 255 for valid data 
    alpha_values = flattened_image[3::4]
    
    land_pixels = np.count_nonzero(alpha_values)
    print("The land area is {} meters squared.".format(land_pixels * area_per_pixel))
    
    return land_pixels * area_per_pixel

In [32]:
def run_tiles(tiles):
    import os 
    import json
    import numpy as np
    import descarteslabs as dl 
    from pprint import pprint
    
    def mask_water(raster):
        shape = raster.shape
        length = raster.size

        # Reshape the matrix to a vector
        x = raster.reshape(length)

        # Slice every 4th element
        # This selects all the nir values from the vector 
        y = x[0::4]

        # Mask the output if less than 60 for NIR 
        # Create a vector of the same size as the NIR vector with all vals 60 for comparison
        sixty = np.ones(len(y))*60

        # Create a vector of boolean values, true when NIR is less than 60     
        z = y < sixty

        # Multiply by 4  - this will extend the bool vector by copying the value 4 times in place
        a = np.repeat(z, 4)

        # Compare the mask to the original raster vector: Returns the value where where false, and '--' where mask is true
        b = np.ma.masked_array(x,a)

        # Give masked nums the value of 0 
        b = np.ma.filled(b,0)

        # Push the output vector into its original 3D shape
        c = b.reshape(shape)
        return c
    
    def get_land_area(image, area_per_pixel):
        # Flatten the rows, columns,and bands into a single array.      
        length =image.size
        flattened_image = image.reshape(length)

        # Select every fourth element, the fourth being the alpha band 
        # with a value of 0 for no valid data and 255 for valid data 
        alpha_values = flattened_image[3::4]

        land_pixels = np.count_nonzero(alpha_values)
        print("The land area is {} meters squared.".format(land_pixels * area_per_pixel))

        return land_pixels * area_per_pixel
   
    new_york = dl.places.shape("north-america_united-states_new-york")
    date = ['2016-06-01','2016-06-30']
    total_land_pixels = 0
    counter = 0
    
    for tile in tiles['features']:
        images = dl.metadata.search(
                                products=["landsat:LC08:PRE:TOAR"],
                                start_time=date[0],
                                end_time=date[1],
                                geom=json.dumps(tile['geometry']),
                                cloud_fraction=0.2,
                                limit = 1000
                                )

        print('Tile #' + str(counter) + '. Number of scenes for this tile: ' + str(len(images['features'])))
        counter += 1
        ids = []
        for image in images['features']:
            ids.append(image['id'])

        arr, meta = dl.raster.ndarray(
            ids,
            bands=['nir', 'swir1', 'red', 'alpha'],
            scales=[[0,6000], [0, 6000], [0, 6000], None],
            data_type='Byte',
            srs = tile['properties']['cs_code'],
            resolution = tile['properties']['resolution'],
            bounds = tile['properties']['outputBounds'],
            cutline = new_york['geometry'])

        arr = arr[16:-16, 16:-16]

        arr = mask_water(arr)
        total_land_pixels += get_land_area(arr,60)

    print('total land pixels: ' + str(total_land_pixels))
    print('square meters: ' + str(total_land_pixels * 3600))
    

## So, you want to apply those predefined python functions to a non-trival sized area?
Enter ***DL Tiles*** and ****DL Tasks***. 

We begin by identifying a geometry of interest via our ```Places``` API. The output is a feature collection of geometries.

In [13]:
new_york = dl.places.shape("north-america_united-states_new-york")
# usa = dl.places.shape("north-america_united-states")

## DL Tiles
Given the large area of interest, it is a good idea to break the region up into smaller, more managable parts. This is where ```DL Tiles``` come in handy. Using the ```DL Tile``` functionality, we can break an area up given ```Places``` API endpoint, a geojson, or a latitude and longitude. 

In [33]:
tiles = dl.raster.dltiles_from_shape(
    resolution= 60.0, 
    tilesize=2048, 
    pad=16, 
    shape=new_york)
pprint(tiles['features'][0])
pprint("Total number of tiles for New York: " + str(len(tiles['features'])))

{
  u'geometry': {
    u'coordinates': [
      [
        [-81.01142542548452, 41.06164192563925],
        [-79.52635307127213, 41.052219545292296],
        [-79.50054542369698, 42.1759483420073],
        [-81.01162560699078, 42.18574788490893],
        ...
      ]
    ],
    u'type': u'Polygon'
  },
  u'properties': {
    u'cs_code': u'EPSG:32617',
    u'key': u'2048:16:60.0:17:0:37',
    u'outputBounds': [499040.0, 4545600.0, 623840.0, 4670400.0],
    u'pad': 16,
    u'resolution': 60.0,
    u'ti': 0,
    u'tilesize': 2048,
    u'tj': 37,
    u'zone': 17
  },
  u'type': u'Feature'
}
'Total number of tiles for New York: 32'


In [28]:
run_tiles(tiles)

Tile #0. Number of scenes for this tile: 6
The land area is 6402960 meters squared.
Tile #1. Number of scenes for this tile: 6
The land area is 5863740 meters squared.
Tile #2. Number of scenes for this tile: 4
The land area is 34540560 meters squared.
Tile #3. Number of scenes for this tile: 4
The land area is 192022740 meters squared.
Tile #4. Number of scenes for this tile: 5
The land area is 14874120 meters squared.
Tile #5. Number of scenes for this tile: 2
The land area is 5438820 meters squared.
Tile #6. Number of scenes for this tile: 2
The land area is 74154540 meters squared.
Tile #7. Number of scenes for this tile: 3
The land area is 26216520 meters squared.
Tile #8. Number of scenes for this tile: 4
The land area is 194534940 meters squared.
Tile #9. Number of scenes for this tile: 2
The land area is 4442520 meters squared.
Tile #10. Number of scenes for this tile: 2
The land area is 75099180 meters squared.
Tile #11. Number of scenes for this tile: 3
The land area is 55825

KeyboardInterrupt: 

## Tasks

In [34]:
from descarteslabs.client.services.tasks import AsyncTasks, as_completed

In [21]:
#There is a one-time initialization to your user-specific resources that you need to run before you can submit jobs:
at = AsyncTasks()
at.create_namespace()

201

In [35]:
at = AsyncTasks()
async_func = at.create_function(
    run_tiles,
    name='demo_scaling',
    image="us.gcr.io/dl-ci-cd/images/tasks/public/geospatial/geospatial-public:latest"
)